In [3]:
from backtesting import Backtest, Strategy
import pandas as pd
from stockstats import wrap 
from BloombergAPI import Bloomberg 
from datetime import datetime
from datetime import date

In [4]:
# To get an RSI with a different period, change the parameter in spx.get('rsi') to spx.get('rsi_period')
# e.g. for a period of 2, do spx.get('rsi_2') - default is 14


spx = pd.read_csv("spx_data.csv")
spx = wrap(spx)
rsi = spx.get('rsi').drop(index=["2007-01-03", "2007-01-04"])
rsi.index = pd.to_datetime(rsi.index, format='mixed').strftime("%m-%d-%Y")
print(rsi)
vix = pd.read_csv("VIX_Futures_Historical.csv", index_col=0)
vix.index = pd.to_datetime(vix.index, format='mixed').strftime("%m-%d-%Y")
vix.index.name = "date"
df = vix.join(rsi, how='inner')
df.drop(["Volume", "Change"],axis=1, inplace=True)

# Combined data
# df.to_csv("VIX_SPX.csv")

df.head()

date
01-05-2007    15.769660
01-08-2007    36.620935
01-09-2007    34.477255
01-10-2007    47.014449
01-11-2007    68.373145
                ...    
07-16-2024    76.366365
07-17-2024    61.150734
07-18-2024    54.660148
07-19-2024    49.528842
07-22-2024    53.154681
Name: rsi, Length: 4415, dtype: float64


,Close,Open,High,Low,rsi
date,,,,,
07-22-2024,15.08,15.83,16.18,15.08,53.154681
07-19-2024,15.92,15.25,16.15,15.23,49.528842
07-18-2024,15.29,14.75,15.50,14.40,54.660148
07-17-2024,13.89,13.05,13.90,13.00,61.150734
07-16-2024,13.16,13.13,13.45,12.95,76.366365


In [5]:

signal = pd.DataFrame(0, index=df.index, columns=["Signal"])
df = pd.concat([df, signal], axis=1)

# RSI thresholds
rsi_90 = df.index[df["rsi"] > 80].to_list()
rsi_30 = df.index[df["rsi"] <= 60].to_list()
df.drop(df.columns[-2], axis=1, inplace=True)

df.loc[rsi_90, "Signal"] = 1
df.loc[rsi_30, "Signal"] = -1
df.index = pd.to_datetime(df.index)

class VIX_LS(Strategy):

    def init(self):
        pass
    def next(self):
        current_signal = self.data.Signal[-1]
        
        trades = self.trades
        def parse_trades(trades):
            for x in trades:
                a = x.entry_time
                b = self.data.index[-1]

                
                delta = b - a


                # Maximum hold period

                if delta.days > 10:
                    x.close()

        
        parse_trades(trades)
        if current_signal == 1:
        
            # size = number of shares, negative for shorts
            self.sell(size=100)
        
        elif current_signal == -1:
            
            self.buy(size=-100)

        # cash = starting balance
bt = Backtest(df, VIX_LS, cash=10000)
stats = bt.run()
print(stats)
bt.plot()

Start                     2009-10-14 00:00:00
End                       2024-07-22 00:00:00
Duration                   5395 days 00:00:00
Exposure Time [%]                   85.991379
Equity Final [$]                      73147.0
Equity Peak [$]                       73552.0
Return [%]                             631.47
Buy & Hold Return [%]              -35.083943
Return (Ann.) [%]                   14.463893
Volatility (Ann.) [%]               73.556468
Sharpe Ratio                         0.196637
Sortino Ratio                        0.344474
Calmar Ratio                         0.205895
Max. Drawdown [%]                  -70.248825
Avg. Drawdown [%]                   -5.632327
Max. Drawdown Duration      669 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                 1882
Win Rate [%]                        60.573858
Best Trade [%]                      42.204724
Worst Trade [%]                    -178.31094
Avg. Trade [%]                    

GridPlot(id='p1261', ...)